<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://raw.githubusercontent.com/Unidata/MetPy/master/metpy/plots/_static/unidata_150x150.png" alt="Unidata Logo" style="height: 98px;">
</div>

<h1>Pythonic Data Analysis</h1>
<h3>Unidata Python Workshop</h3>

<div style="clear:both"></div>
</div>

<hr style="height:2px;">

<div style="float:right; width:250 px"><img src="http://matplotlib.org/_images/date_demo.png" alt="METAR" style="height: 300px;"></div>


## Overview:

* **Teaching:** 45 minutes
* **Exercises:** 45 minutes

### Questions
1. How can we employ Python language features to make complicated analysis require less code?
1. How can we make multi panel plots?
1. What can be done to eliminate repeated code that operates on sequences of objects?
1. How can functions be used to encapsulate calculations and behaviors?

### Objectives
1. <a href="#basicfunctionality">From the Time Series Plotting Episode</a>
1. <a href="#multipanel">Multi-panel plots</a>
1. <a href="#iteration">Iteration and Enumeration</a>
1. <a href="#functions">Functions</a>
1. <a href="#plottingiteration">Plotting with Iteration</a>
1. <a href="#multifile">Plotting Multiple Files</a>

<a name="basicfunctionality"></a>
## From Time Series Plotting Episode
Here's the basic set of imports and data reading functionality that we established in the [Basic Time Series Plotting](../Time_Series/Basic Time Series Plotting.ipynb) notebook.

In [ ]:
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, DayLocator

%matplotlib inline

# Register pandas converters with matplotlib
from pandas.tseries import converter
converter.register()

In [ ]:
fname='41056.txt'

col_names = ['year', 'month', 'day', 'hour', 'minute', 'wind_direction', 'wind_speed',
             'wind_gust', 'wave_height', 'dominant_wave_period', 'average_wave_period',
             'dominant_wave_direction', 'pressure', 'temperature', 'water_temperature', 'dewpoint',
             'visibility', '3hr_pressure_tendency', 'water_level_above_mean']
    
df = pd.read_fwf(fname, skiprows=2, na_values='MM', names=col_names)
df['time'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute']])
df.drop(['year', 'month', 'day', 'hour', 'minute'], axis='columns', inplace=True)
idx = (df.time >= datetime(2017, 9, 6)) & (df.time <= datetime(2017, 9, 8))
df = df[idx]
df.reset_index(drop=True, inplace=True)

<a name="multipanel"></a>
## Multi-panel Plots

Often we wish to create figures with multiple panels of data. It's common to separate variables of different types into these panels. We also don't want to create each panel as an individual figure and combine them in a tool like Illustrator - imagine having to do that for hundreds of plots!

We can use the `subplots` method to specify a number of rows and columns of plots in our figure:

In [ ]:
# ShareX means that the axes will share range, ticking, etc. for the x axis
fig, (ax1, ax2) = plt.subplots(1, 2, sharex=True, figsize=(18, 6))

# Panel 1
ax1.plot(df.time, df.wind_speed, color='tab:orange', label='Windspeed')
ax1.set_xlabel('Time')
ax1.set_ylabel('Speed')
ax1.set_title('Measured Winds')
ax1.legend(loc='upper left')
ax1.grid(True)

# Not repeated only by sharing x
ax1.xaxis.set_major_formatter(DateFormatter('%m/%d'))
ax1.xaxis.set_major_locator(DayLocator())

# Panel 2
ax2.plot(df.time, df.pressure, color='black', label='Pressure')
ax2.set_xlabel('Time')
ax2.set_ylabel('hPa')
ax2.set_title('Atmospheric Pressure')
ax2.legend(loc='upper left')
ax2.grid(True)

plt.suptitle('Buoy 41056 Data', fontsize=24)

So even with the sharing of axis information, there's still a lot of repeated code. This current version with just two parameters might still be ok, but:

- What if we had more data being plotted on each axes?
- What if we had many subplots?
- What if we wanted to change one of the parameters?
- What if we wanted to plot data from different files on the same plot?

<a name="iteration"></a>
### Iteration and Enumeration
Iterating over lists is a very useful tool to reduce the amount of repeated code you write. We're going to start out by iterating over a single list with a `for` loop. Unlike C or other common scientific languages, Python 'knows' how to iterate over certain objects without you needing to specify an index variable and do the book keeping on that.

In [ ]:
my_list = ['2001 A Space Obyssey',
           'The Princess Bride',
           'Monty Python and the Holy Grail']

for item in my_list:
    print(item)

Using the `zip` function we can even iterate over multiple lists at the same time with ease:

In [ ]:
my_other_list = ['I\'m sorry, Dave. I\'m afraid I can\'t do that.',
                 'My name is Inigo Montoya.',
                 'It\'s only a flesh wound.']

for item in zip(my_list, my_other_list):
    print(item)

That's really handy, but needing to access each part of each item with an index like `item[0]` isn't very flexible, requires us to remember the layout of the item, and isn't best practice. Instead we can use Python's unpacking syntax to make things nice an intuitive.

In [ ]:
for reference, quote in zip(my_list, my_other_list):
    print(reference, '-', quote)

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
    <li>Make two new lists named <code>plot_variables</code> and <code>plot_names</code>. Populate them
        with the variable name and plot label string for windspeed and pressure.</li>
    <li>Using the unpacking syntax, write a for loop that prints a sentence describing the action
        that would be taken (i.e. Plotting variable wind_speed as Windspeed</li>
    </ul>
</div>

In [ ]:
# Your code goes here


In [ ]:
# %load solutions/pythonic_parameter_iteration.py

`zip` can also be used to unzip as a sequence can be passed as a set of positional arguments.

In [ ]:
z = zip(my_list, my_other_list)
print(list(zip(*z)))

We can use the `enumerate` function to 'count through' an iterable object as well. This can be useful when placing figures in certain rows/columns or when a counter is needed.

In [ ]:
for i, quote in enumerate(my_other_list):
    print(i, ' - ', quote)

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
    <li>Combine what you've learned about enumeration and iteration to produce the following output:</li>
    </ul>
    <code>0  -  2001 A Space Obyssey  -  I'm sorry, Dave. I'm afraid I can't do that.
1  -  The Princess Bride  -  My name is Inigo Montoya.
2  -  Monty Python and the Holy Grail  -  It's only a flesh wound.</code>
</div>

In [ ]:
# Your code goes here


In [ ]:
# %load solutions/pythonic_enumeration.py

<a name="functions"></a>
### Functions
You're probably already familiar with Python functions, but here's a quick refresher. Functions are used to house blocks of code that we can run repeatedly. Paramters are given as inputs, and values are returned from the function to where it was called. In the world of programming you can think of functions like paragraphs, they encapsulate a complete idea/process.

Let's define a simple function that returns a value:

In [ ]:
def silly_add(a, b):
    return a + b

We've re-implemented add which isn't incredibly exiciting, but that could be hundreds of lines of a numerical method, making a plot, or some other task. Using the function is simple:

In [ ]:
result = silly_add(3, 4)
print(result)

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
    <li>Write a function that returns powers of 2. (i.e. calling <code>myfunc(4)</code> returns 2^4)</li>
    <li>**Bonus**: Using for loop iteration, print all powers of 2 from 0 to 24.</li>
    </ul>
</div>

In [ ]:
# Your code goes here


In [ ]:
# %load solutions/pythonic_functions.py

<a name="plottingiteration"></a>
### Plotting with Iteration
Now let's bring what we've learned about iteration to bear on the problem of plotting. We'll start with a basic example and roll into a more involved system at the end.

To begin, let's make an arbitrary number of plots in a single row:

In [ ]:
# A list of names of variables we want to plot
plot_variables = ['wind_speed', 'pressure']

# Make our figure, now choosing number of subplots based on length of variable name list
fig, axes = plt.subplots(1, len(plot_variables), sharex=True, figsize=(18, 6))

# Loop over the list of subplots and names together
for ax, var_name in zip(axes, plot_variables):
    
    ax.plot(df.time, df[var_name])

    # Set label/title based on variable name--no longer hard-coded
    ax.set_ylabel(var_name)
    ax.set_title('Buoy {}'.format(var_name))

    # Set up our formatting--note lack of repetition
    ax.grid(True)
    ax.set_xlabel('Time')
    ax.xaxis.set_major_formatter(DateFormatter('%m/%d'))
    ax.xaxis.set_major_locator(DayLocator())

It's a step forward, but we've lost a lot of formatting information. The lines are both blue, the labels as less ideal, and the title just uses the variable name. We can use some of Python's features like dictionaries, functions, and string manipulation to help improve the versatility of the plotter.

To start out, let's get the line color functionality back by using a Python dictionary to hold that information. Dictionaries can hold any data type and allow you to access that value with a key (hence the name key-value pair). We'll use the variable name for the key and the value will be the color of line to plot.

In [ ]:
colors = {'wind_speed': 'tab:orange', 'wind_gust': 'tab:olive', 'pressure': 'black'}

To access the value, just access that element of the dictionary with the key.

In [ ]:
colors['pressure']

Now let's apply that to our plot. We'll use the same code from the previous example, but now look up the line color in the dictionary.

In [ ]:
fig, axes = plt.subplots(1, len(plot_variables), sharex=True, figsize=(18, 6))

for ax, var_name in zip(axes, plot_variables):

    # Grab the color from our dictionary and pass it to plot()
    color = colors[var_name]
    ax.plot(df.time, df[var_name], color)

    ax.set_ylabel(var_name)
    ax.set_title('Buoy {}'.format(var_name))

    ax.grid(True)
    ax.set_xlabel('Time')
    ax.xaxis.set_major_formatter(DateFormatter('%m/%d'))
    ax.xaxis.set_major_locator(DayLocator())

That's already much better. We need to be able to plot multiple variables on the wind speed/gust plot though. In this case, we'll allow a list of variables for each plot to be given and iterate over them. We'll store this in a list of lists. Each plot has its own list of variables!

In [ ]:
plot_variables = [['wind_speed', 'wind_gust'], ['pressure']]

fig, axes = plt.subplots(1, len(plot_variables), sharex=True, figsize=(18, 6))

for ax, var_names in zip(axes, plot_variables):

    for var_name in var_names:
        # Grab the color from our dictionary and pass it to plot()
        color = colors[var_name]
        ax.plot(df.time, df[var_name], color)

    ax.set_ylabel(var_name)
    ax.set_title('Buoy {}'.format(var_name))

    ax.grid(True)
    ax.set_xlabel('Time')
    ax.xaxis.set_major_formatter(DateFormatter('%m/%d'))
    ax.xaxis.set_major_locator(DayLocator())

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
    <li>Create a dictionary of linestyles in which the variable name is the key and the linestyle is the value.</li>
    <li>Use that dictionary to modify the code below to plot the lines with the styles you specified.</li>
    </ul>
</div>

In [ ]:
# Create your linestyles dictionary and modify the code below

fig, axes = plt.subplots(1, len(plot_variables), sharex=True, figsize=(18, 6))

for ax, var_names in zip(axes, plot_variables):

    for var_name in var_names:
        # Grab the color from our dictionary and pass it to plot()
        color = colors[var_name]
        ax.plot(df.time, df[var_name], color)

    ax.set_ylabel(var_name)
    ax.set_title('Buoy {}'.format(var_name))

    ax.grid(True)
    ax.set_xlabel('Time')
    ax.xaxis.set_major_formatter(DateFormatter('%m/%d'))
    ax.xaxis.set_major_locator(DayLocator())

In [ ]:
# %load solutions/pythonic_linestyles.py

We're almost back to where to started, but in a much more versatile form! We just need to make the labels and titles look nice. To do that, let's write a function that uses some string manipulation to clean up the variable names and give us an axis/plot title and legend label.

In [ ]:
def format_varname(varname):
    parts = varname.split('_')
    title = parts[0].title()
    label = varname.replace('_', ' ').title()
    return title, label

In [ ]:
fig, axes = plt.subplots(1, len(plot_variables), sharex=True, figsize=(18, 6))

for ax, var_names in zip(axes, plot_variables):

    for var_name in var_names:
        
        title, label = format_varname(var_name)
        
        color = colors[var_name]
        linestyle = linestyles[var_name]
        ax.plot(df.time, df[var_name], color, linestyle=linestyle, label=label)

    ax.set_ylabel(title)
    ax.set_title('Buoy {}'.format(title))

    ax.grid(True)
    ax.set_xlabel('Time')
    ax.xaxis.set_major_formatter(DateFormatter('%m/%d'))
    ax.xaxis.set_major_locator(DayLocator())
    ax.legend(loc='upper left')

<a href="#top">Top</a>
<hr style="height:2px;">

<a name="multifile"></a>
## Plotting Multiple Files

Finally, let's plot data for two buoys on the same figure by iterating over a list of file names. We can use enumerate to plot each file on a new row of the figure. We will also create a function to read in the buoy data and avoid all of that repeated code.

In [ ]:
def read_buoy_data(fname):
    """Handy function to read and cleanup the buoy data files."""
    col_names = ['year', 'month', 'day', 'hour', 'minute', 'wind_direction', 'wind_speed',
             'wind_gust', 'wave_height', 'dominant_wave_period', 'average_wave_period',
             'dominant_wave_direction', 'pressure', 'temperature', 'water_temperature', 'dewpoint',
             'visibility', '3hr_pressure_tendency', 'water_level_above_mean']
    
    df = pd.read_fwf(fname, skiprows=2, na_values='MM', names=col_names)

    df['time'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute']])

    # Using inplace means the return is None and the dataframe is simply modified.
    df.drop(['year', 'month', 'day', 'hour', 'minute'], axis='columns', inplace=True)
    
    idx = (df.time >= datetime(2017, 9, 6)) & (df.time <= datetime(2017, 9, 8))
    df = df[idx]
    
    df.reset_index(drop=True, inplace=True)
    
    return df

In [ ]:
files = ['41056.txt', '41052.txt']

fig, axes = plt.subplots(len(files), len(plot_variables), sharex=True, figsize=(14, 10))

for row, filename in enumerate(files):
    df = read_buoy_data(filename)
    

    for col, var_names in enumerate(plot_variables):
        ax = axes[row,col]
        for var_name in var_names:
            title, label = format_varname(var_name)
            color = colors[var_name]
            linestyle = linestyles[var_name]
            ax.plot(df.time, df[var_name], color, linestyle=linestyle, label=label)

        ax.set_ylabel(title)
        ax.set_title('Buoy {} {}'.format(filename.split('.')[0], title))

        ax.grid(True)
        ax.set_xlabel('Time')
        ax.xaxis.set_major_formatter(DateFormatter('%m/%d'))
        ax.xaxis.set_major_locator(DayLocator())

### A quick lesson on troubleshooting
Looking at the plot of 41052 something is wrong. The winds never go above 9 m/s. We know that can't be right. There could be a problem in the data or it could be our data reading. Let's open the file and take a look in the time range of 9/20-9/21:

<code>2017 09 20 08 30 100 21.0 29.0    MM    MM    MM  MM  993.7  26.6    MM    MM   MM   MM    MM
2017 09 20 08 20 100 20.0 25.0    MM    MM    MM  MM  994.1  26.8    MM    MM   MM   MM    MM
2017 09 20 08 00 100 20.0 28.0   6.2    11    MM 139  993.2  26.6    MM    MM   MM -1.8    MM
2017 09 20 07 10 100 21.0 26.0   7.9    MM    MM 136  992.5  26.4    MM    MM   MM   MM    MM
2017 09 20 07 00  90 20.0 25.0   7.9    11    MM 136  992.5  26.4    MM    MM   MM -5.1    MM
2017 09 20 06 50  90 20.0 26.0    MM    MM    MM  MM  992.6  26.7    MM    MM   MM   MM    MM
2017 09 20 06 40  90 20.0 26.0    MM    MM    MM  MM  992.7  26.8    MM    MM   MM   MM    MM
2017 09 20 06 30  90 21.0 26.0    MM    MM    MM  MM  992.7  26.6    MM    MM   MM   MM    MM
2017 09 20 06 20  90 21.0 26.0    MM    MM    MM  MM  993.1  26.2    MM    MM   MM   MM    MM
2017 09 20 06 00  90 20.0 27.0   6.7    11    MM 133  993.1  26.6    MM    MM   MM -5.4    MM
2017 09 20 05 30  80 20.0 25.0    MM    MM    MM  MM  993.6  26.7    MM    MM   MM   MM    MM
2017 09 20 05 20  80 19.0 25.0    MM    MM    MM  MM  994.3  26.8    MM    MM   MM   MM    MM
2017 09 20 05 00  70 18.0 24.0   5.5    12    MM 129  995.0  26.9    MM    MM   MM -5.5    MM
2017 09 20 04 10  70 17.0 25.0   5.7    MM    MM 125  997.0  26.7    MM    MM   MM   MM    MM</code>

Hmmm... There are values above 9 m/s there! Looks like we are not parsing the file correctly. Taking a peek at the pandas documentation gives us a hint:

<code>colspecs : list of pairs (int, int) or 'infer'. optional
    A list of pairs (tuples) giving the extents of the fixed-width
    fields of each line as half-open intervals (i.e.,  [from, to[ ).
    String value 'infer' can be used to instruct the parser to try
    detecting the column specifications from the first 100 rows of
    the data which are not being skipped via skiprows (default='infer').
widths : list of ints. optional
    A list of field widths which can be used instead of 'colspecs' if
    the intervals are contiguous.</code>
    
The fixed width format is being inferred from the first 100 lines of the file. If there are no values over 9 m/s there, the parsing will likley not ever pick up the 10's place digits. We can fix this by counting the width of the columns and manually telling that to pandas. Looking at the file we see the data are 'right justified', so the count should start at the end of the previous element and end at the termination of the current element.

In [ ]:
widths = [4, 3, 3, 3, 3, 4, 5, 5, 6, 6, 6, 4, 7, 6, 6, 6, 5, 5, 6]

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
    <li>Add the widths to the <code>read_buoy_data</code> function and retry the plot again. (You can simply modify the function above and re-run the plot cell.)</li>
    </ul>
</div>

In [ ]:
# %load solutions/pythonic_troubleshooting.py

<a href="#top">Top</a>
<hr style="height:2px;">